<a href="https://colab.research.google.com/github/laskari/END-Program/blob/main/Model_1_Wiki_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sequence - Squence Model on Wiki Question and Answer Dataset


In [1]:
import pandas as pd
print('Train Data')
data_train = pd.read_csv('/content/drive/MyDrive/END School of AI/Datasets/END Datasets/WikiQACorpus/WikiQA.tsv', sep='\t')
data_df = data_train[data_train['Label'] ==1]
data_new = data_df[['Question','Sentence']]
data_new.reset_index(drop=True, inplace=True)

print(data_new.shape)

import pandas as pd
print('Validation Data')
data_dev = pd.read_csv('/content/drive/MyDrive/END School of AI/Datasets/END Datasets/WikiQACorpus/WikiQA-dev.tsv', sep='\t')
data_1 = data_dev[data_dev['Label'] ==1]
data_dev = data_1[['Question','Sentence']]
data_dev.reset_index(drop=True, inplace=True)
print(data_dev.shape)

Train Data
(1469, 2)
Validation Data
(140, 2)


In [2]:
data_new.head(3)

,Question,Sentence
0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,"As such, African immigrants are to be distingu..."
1,how are glacier caves formed?,A glacier cave is a cave formed within the ice...
2,how a water pump works,Pumps operate by some mechanism (typically rec...


In [3]:
data_dev.head(2)

,Question,Sentence
0,"how big is bmc software in houston, tx","Employing over 6,000, BMC is often credited wi..."
1,"how big is bmc software in houston, tx","For 2011, the company recorded an annual reven..."


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import random
from torchtext import data 
import math
import time

In [5]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

spacy_en = spacy.load('en')

In [6]:
def tokenize_en(text):
  return [tok.text for tok in spacy_en.tokenizer(text)]

In [7]:
Question = Field(tokenize = 'spacy',
            init_token='<sos>',
            eos_token ='<eos',
            lower = True)

Sentence = Field(tokenize = 'spacy',
            init_token='<sos>',
            eos_token ='<eos',
            lower = True)

fields = [('Question', Question),('Sentence',Sentence)]

In [8]:
example_train = [data.Example.fromlist([data_new.Question[i],data_new.Sentence[i]], fields) for i in range(data_new.shape[0])] 

example_dev = [data.Example.fromlist([data_dev.Question[i],data_dev.Sentence[i]], fields) for i in range(data_dev.shape[0])]
#train_data, valid_data, test_data = data_new.splits(fields = (Question, Sentence))

In [9]:
train_Dataset = data.Dataset(example_train, fields)
dev_Dataset = data.Dataset(example_dev, fields)
print(len(train_Dataset))
print(len(dev_Dataset))

1469
140


In [10]:
vars(train_Dataset.examples[0])

{'Question': ['how',
  'african',
  'americans',
  'were',
  'immigrated',
  'to',
  'the',
  'us'],
 'Sentence': ['as',
  'such',
  ',',
  'african',
  'immigrants',
  'are',
  'to',
  'be',
  'distinguished',
  'from',
  'african',
  'american',
  'people',
  ',',
  'the',
  'latter',
  'of',
  'whom',
  'are',
  'descendants',
  'of',
  'mostly',
  'west',
  'and',
  'central',
  'africans',
  'who',
  'were',
  'involuntarily',
  'brought',
  'to',
  'the',
  'united',
  'states',
  'by',
  'means',
  'of',
  'the',
  'historic',
  'atlantic',
  'slave',
  'trade',
  '.']}

In [11]:
Question.build_vocab(train_Dataset)
Sentence.build_vocab(train_Dataset)

In [12]:
print('Size of Question vocab : ', len(Question.vocab))
print('Size of Answer vocab : ', len(Sentence.vocab))

Size of Question vocab :  2524
Size of Answer vocab :  8838


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [14]:
train_iterator, valid_iterator = data.BucketIterator.splits((train_Dataset, dev_Dataset), batch_size = 32,  sort = False, device = device)

In [15]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [16]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [17]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [18]:
INPUT_DIM = len(Question.vocab)
OUTPUT_DIM = len(Sentence.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [19]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2524, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(8838, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=8838, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 17,382,022 trainable parameters


In [21]:
optimizer = optim.Adam(model.parameters())

TRG_PAD_IDX = Sentence.vocab.stoi[Sentence.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [22]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.Question
        trg = batch.Sentence
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [23]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.Question
            trg = batch.Sentence

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [24]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 12s
	Train Loss: 7.535 | Train PPL: 1872.483
	 Val. Loss: 6.838 |  Val. PPL: 932.278
Epoch: 02 | Time: 0m 12s
	Train Loss: 6.766 | Train PPL: 867.482
	 Val. Loss: 6.711 |  Val. PPL: 821.717
Epoch: 03 | Time: 0m 12s
	Train Loss: 6.577 | Train PPL: 718.113
	 Val. Loss: 6.741 |  Val. PPL: 846.549
Epoch: 04 | Time: 0m 12s
	Train Loss: 6.437 | Train PPL: 624.617
	 Val. Loss: 6.593 |  Val. PPL: 729.701
Epoch: 05 | Time: 0m 12s
	Train Loss: 6.313 | Train PPL: 551.621
	 Val. Loss: 6.570 |  Val. PPL: 713.424
Epoch: 06 | Time: 0m 12s
	Train Loss: 6.200 | Train PPL: 492.601
	 Val. Loss: 6.526 |  Val. PPL: 682.634
Epoch: 07 | Time: 0m 12s
	Train Loss: 6.101 | Train PPL: 446.392
	 Val. Loss: 6.428 |  Val. PPL: 618.703
Epoch: 08 | Time: 0m 12s
	Train Loss: 5.973 | Train PPL: 392.783
	 Val. Loss: 6.340 |  Val. PPL: 566.867
Epoch: 09 | Time: 0m 12s
	Train Loss: 5.826 | Train PPL: 338.835
	 Val. Loss: 6.288 |  Val. PPL: 538.187
Epoch: 10 | Time: 0m 12s
	Train Loss: 5.671 | Train PP